# Set of those contracts from API that match filter

In [1]:
import json
with open('output_data.json', 'r') as f:
    data = json.load(f)


filtered_titles_set = {entry['item']['title'] for entry in data['noticeList']}

filtered_titles_set

{'A DPS for sustainable and ethically sourced goods and services, delivering social value in local communities',
 'AISI Challenge Fund Grant',
 'Business Support Services DPS',
 'CA15822 - Askham Bryan College - Farm Management',
 'COV - Monitoring and Evaluation of School Improvement Activity',
 'CP2545-24 School Meal Vouchers',
 'Cleaning contract Public Toilets - Lynton &amp; Lynmouth',
 'Climate Change Pull Facility (CIPF) - Evaluation Partner',
 'Climate Innovation Pull Facility - Facility Manager - Invitation to Submit Initial Tenders',
 'Commissioned Rehabilitative Services  - Expression of Interest',
 'Communication and Print DPS Framework',
 'Communications Marketplace',
 'Construction Professional Services DPS',
 'Construction Works and Associated Services including P24 Healthcare and Offsite Solutions',
 'Consultancy Services Dynamic Purchasing System (DPS)',
 'Consultancy Support',
 'DPS - Dynamic Supplier List: Asset Solutions',
 'Defence Medical Services Research Steering

# Get set of jobs already on Opps link

In [2]:
import requests

def get_all_job_titles(api_key, base_url="https://opps-link.com/api/jobs"):
    """
    Retrieve ALL active job titles from the API by paginating through all results
    
    Args:
        api_key (str): Your API key
        base_url (str): Base API URL
        
    Returns:
        set: Set of ALL active job titles across all pages
    """
    job_titles = set()
    page = 1
    limit = 100  # Max per API docs
    total_jobs = 0
    
    print("Starting to collect all job titles...")
    
    while True:
        params = {
            'page': page,
            'limit': limit,
            'active': 1,
            'api_key': api_key
        }
        
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            
            # Check if we got jobs
            current_jobs = data.get('jobs', [])
            if not current_jobs:
                break  # No more jobs
                
            # Add all titles from this page
            for job in current_jobs:
                job_titles.add(job['title'])
            
            total_jobs += len(current_jobs)
            
            # Progress reporting
            print(f"Processed page {page} - found {len(current_jobs)} jobs (total: {total_jobs})")
            
            # Check if we should continue
            if len(current_jobs) < limit:
                break  # Last page
                
            page += 1
            
        except requests.exceptions.RequestException as e:
            print(f"Error on page {page}: {e}")
            break
            
    print(f"\nFinished! Collected {len(job_titles)} unique titles from {total_jobs} jobs across {page} pages")
    return job_titles

# Usage
api_key = "37c597e7bb52d26099ede8b8aa43b270"
all_titles = get_all_job_titles(api_key)

all_titles=set(all_titles)

Starting to collect all job titles...


Processed page 1 - found 100 jobs (total: 100)


Processed page 2 - found 100 jobs (total: 200)


Processed page 3 - found 87 jobs (total: 287)

Finished! Collected 287 unique titles from 287 jobs across 3 pages


# Get all existing employers and employer ids

In [3]:
import requests

def get_all_employers(api_key, base_url="https://opps-link.com/api/employers"):
    """
    Retrieve ALL employers with their IDs from the API by paginating through all results
    
    Args:
        api_key (str): Your API key
        base_url (str): Base API URL
        
    Returns:
        dict: Dictionary mapping employer names (lowercase) to IDs
        list: List of all employer dictionaries
    """
    employers_dict = {}  # This will now be {name_lowercase: id}
    employers_list = []
    page = 1
    limit = 100  # Adjust based on API's max limit
    total_processed = 0
    
    print("Starting to collect all employers...")
    
    while True:
        params = {
            'page': page,
            'limit': limit,
            'api_key': api_key
        }
        
        try:
            response = requests.get(base_url, params=params)
            response.raise_for_status()
            data = response.json()
            
            # Check if we got employers
            current_employers = data.get('employers', [])
            if not current_employers:
                break  # No more employers
                
            # Process employers from this page
            for employer in current_employers:
                employer_id = employer['id']
                employer_name = employer['company_name']
                # Use lowercase name for case-insensitive matching
                employers_dict[employer_name.lower()] = employer_id
                employers_list.append(employer)
            
            total_processed += len(current_employers)
            
            # Progress reporting
            print(f"Processed page {page} - found {len(current_employers)} employers (total: {total_processed})")
            
            # Check if we should continue
            if len(current_employers) < limit:
                break  # Last page
                
            page += 1
            
        except requests.exceptions.RequestException as e:
            print(f"Error on page {page}: {e}")
            break
            
    print(f"\nFinished! Collected {len(employers_dict)} unique employers across {page} pages")
    return employers_dict, employers_list

# Usage
api_key = "37c597e7bb52d26099ede8b8aa43b270"
employers_dict, employers_list = get_all_employers(api_key)

# Now employers_dict is {name_lowercase: id} which is what you need
print(f"Sample employer: {list(employers_dict.items())[0]}")

Starting to collect all employers...


Processed page 1 - found 100 employers (total: 100)


Processed page 2 - found 100 employers (total: 200)


Processed page 3 - found 100 employers (total: 300)


Processed page 4 - found 100 employers (total: 400)


Processed page 5 - found 92 employers (total: 492)

Finished! Collected 485 unique employers across 5 pages
Sample employer: ('british council', 9)


In [4]:
employers_dict

{'british council': 9,
 'defra - department for environment, food and rural affairs': 10,
 'european parliament - dg for communication (comm)': 14,
 'european committee of the regions': 15,
 'european commission - eurostat': 16,
 'european union aviation safety agency (easa)': 18,
 'european food safety authority (efsa)': 19,
 'united nations development program, panama (undp)': 20,
 'northern ireland housing executive': 21,
 'cpd - supplies and services division': 22,
 'european centre for disease prevention and control (ecdc)': 23,
 'european commission - dg for neighbourhood and enlargement negotiations (dg near)': 33,
 'european commission - european climate, infrastructure and environment executive agency (cinea)': 34,
 'european commission - dg for defence industry and space (defis)': 35,
 'european investment bank (eib)': 37,
 'european commission - dg for education, youth, sport and culture (eac)': 38,
 'european commission - service for foreign policy instruments (fpi)': 39,
 

# Attempt 1 

In [5]:
import requests
from datetime import datetime, timezone
from parsel import Selector
import json
import html
import re
from dateutil import parser

# Configuration
JOB_API_URL = "https://opps-link.com/api/jobs"
EMPLOYER_API_URL = "https://opps-link.com/api/employers"
API_KEY = "37c597e7bb52d26099ede8b8aa43b270"
HEADERS = {"Content-Type": "application/json"}

# Load existing data
with open('output_data.json', 'r') as f:
    json_data = json.load(f)

# Initialize sets/dicts from your existing data
filtered_titles_set = {entry['item']['title'] for entry in json_data['noticeList']}
contract_titles = set()  # Should be populated with existing job titles from your site
employer_dict = {}       # Should be populated with existing employers {name: id}

def clean_description(description):
    """Clean and format the description text."""
    if not description:
        return "Not Disclosed"
    cleaned = html.unescape(description)
    cleaned = cleaned.replace('\r\n', ' ').replace('\n', ' ')
    return ' '.join(cleaned.split()).strip()

def create_employer(company_name):
    """Create a new employer and return their ID"""
    # Generate a standardized email and password
    clean_name = re.sub(r'[^a-z0-9_]', '', company_name.lower().replace(' ', '_'))
    email = f"{clean_name}@opps-link.com"
    password = f"{clean_name}_123!"  # Simple password that meets requirements
    
    payload = {
        "api_key": API_KEY,
        "email": email,
        "password": password,  # Added required password field
        "company_name": company_name,
        "company_description": f"<p>{company_name}</p>",
        "active": 1,
        "featured": 0,
        "registration_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "full_name": company_name,  # Some APIs require this
        "location": "United Kingdom",  # Default location
        "website": ""  # Empty but required field
    }
    
    try:
        response = requests.post(EMPLOYER_API_URL, json=payload, headers=HEADERS)
        if response.status_code == 201:
            employer_id = response.json().get('id')
            print(f"✅ Created new employer: {company_name} (ID: {employer_id})")
            return employer_id
        else:
            print(f"❌ Failed to create employer {company_name}: {response.text}")
            return None
    except Exception as e:
        print(f"🔥 Error creating employer: {str(e)}")
        return None

def create_job(job_data):
    """Create a new job posting"""
    try:
        response = requests.post(JOB_API_URL, json=job_data, headers=HEADERS)
        if response.status_code == 201:
            job_id = response.json().get('id')
            print(f"✅ Successfully posted job: {job_data['title']} (ID: {job_id})")
            return True
        else:
            print(f"❌ Failed to post job {job_data['title']}: {response.text}")
            return False
    except Exception as e:
        print(f"🔥 Error posting job: {str(e)}")
        return False

def process_contracts():
    base_url = "https://www.contractsfinder.service.gov.uk/Search/Results?page="
    
    for page_number in range(1, 5):
        url = base_url + str(page_number)
        try:
            response = requests.get(url)
            response.raise_for_status()
            sel = Selector(text=response.text)
            contract_nodes = sel.xpath('//div[@class="search-result"]')
            
            if not contract_nodes:
                break
                
            for node in contract_nodes:
                name = node.xpath('.//div[@class="search-result-header"]/@title').get()
                
                if not name or name not in filtered_titles_set or name in all_titles:
                    continue
                
                additional_fields = title_to_details.get(name, {})
                client_name = additional_fields.get('client', 'Not Disclosed')
                client_key = client_name.lower()
                
                print(f"Processing job: {name} | Employer: {client_name}")
                
                employer_id = employers_dict.get(client_key)
                
                if employer_id is None:
                    print(f"Employer '{client_name}' not found, attempting to create...")
                    employer_id = create_employer(client_name)
                    if employer_id:
                        employers_dict[client_key] = employer_id
                    else:
                        continue
                else:
                    print(f"Found existing employer: {client_name} (ID: {employer_id})")
                
                contract_value = node.xpath('.//strong[contains(text(), "Contract value")]/following-sibling::text()').get()
                contract_link = node.xpath('.//div[@class="search-result-header"]//a/@href').get()
                
                # Handle deadline date - use same value for both expiration_date and application deadline
                deadline_date = additional_fields.get('deadline_date')
                if deadline_date:
                    try:
                        # Parse the date string to ensure it's valid
                        parsed_date = parser.parse(deadline_date)
                        # Format as YYYY-MM-DD for API compatibility
                        formatted_date = parsed_date.strftime("%Y-%m-%d")
                    except:
                        formatted_date = '2025-12-31'  # Default fallback
                else:
                    formatted_date = '2025-12-31'  # Default fallback
                
                job_data = {
                    "api_key": API_KEY,
                    "active": 1,
                    "featured": 0,
                    "title": name,
                    "employer_id": employer_id,
                    "description": additional_fields.get('description', ''),
                    "how_to_apply": "contact@example.com",
                    "activation_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "expiration_date": formatted_date,  # Uses deadlineDate
                    "location": additional_fields.get('region', 'Not Specified'),
                    "custom_fields": [
                        {"name": "Contract Value", "value": contract_value.strip() if contract_value else 'Not Disclosed'},
                        {"name": "Contract Link", "value": contract_link.strip() if contract_link else ''},
                        {
                            "name": "Application deadline", 
                            "value": formatted_date,  # Same as expiration_date
                            "type": "date"
                        }
                    ]
                }
                
                if create_job(job_data):
                    contract_titles.add(name)
                    
        except Exception as e:
            print(f"Error processing page {page_number}: {str(e)}")
            continue


# Create title_to_details mapping
title_to_details = {}
for entry in json_data['noticeList']:
    item = entry['item']
    title = item['title']
    title_to_details[title] = {
        'client': item.get('organisationName', 'Not Disclosed'),
        'description': clean_description(item.get('description')),
        'deadline_date': item.get('deadlineDate', '2025-12-31 23:59:59'),
        'region': item.get('regionText', item.get('region', 'Not Disclosed')),
        'cpv_codes': item.get('cpvCodes', 'Not Available'),
        'notice_status': item.get('noticeStatus', 'Unknown')
    }

# Start processing
process_contracts()
print("Job processing complete!")

Job processing complete!


# EU contracts attempt 1 

In [6]:
def create_job(job_data):
    """Create a new job posting."""
    try:
        response = requests.post("https://opps-link.com/api/jobs", json=job_data, headers=HEADERS)
        if response.status_code == 201:
            job_id = response.json().get('id')
            print(f"✅ Successfully posted job: {job_data['title']} (ID: {job_id})")
            return True
        else:
            print(f"❌ Failed to post job {job_data['title']}: {response.status_code} - {response.text}")
            return False
    except Exception as e:
        print(f"🔥 Error posting job {job_data['title']}: {str(e)}")
        return False
    
def create_employer(company_name):
    """Create a new employer and return their ID."""
    clean_name = re.sub(r'[^a-z0-9_]', '', company_name.lower().replace(' ', '_'))
    email = f"{clean_name}@opps-link.com"
    password = f"{clean_name}_123!"

    payload = {
        "api_key": API_KEY_OPPSLINK,
        "email": email,
        "password": password,
        "company_name": company_name,
        "company_description": f"<p>{company_name}</p>",
        "active": 1,
        "featured": 0,
        "registration_date": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "full_name": company_name,
        "location": "EU",
        "website": ""
    }

    try:
        response = requests.post("https://opps-link.com/api/employers", json=payload, headers=HEADERS)
        if response.status_code == 201:
            employer_id = response.json().get('id')
            print(f"✅ Created employer: {company_name} (ID: {employer_id})")
            return employer_id
        else:
            print(f"❌ Failed to create employer {company_name}: {response.status_code} - {response.text}")
            return None
    except Exception as e:
        print(f"🔥 Error creating employer {company_name}: {str(e)}")
        return None



In [7]:
import httpx
import json
import time
from datetime import datetime, timezone
import html
import re
import requests

# Constants
API_KEY_OPPSLINK = "37c597e7bb52d26099ede8b8aa43b270"
HEADERS = {"Content-Type": "application/json"}
EU_API_KEY = "844692394d814535a9ed3a37f0ab01e1"
EU_API_URL = "https://tedweb.api.ted.europa.eu/private-search/api/v1/notices/search"

# Step 1: Get all employers and job titles from OppsLink
employers_dict, employers_list = get_all_employers(API_KEY_OPPSLINK)
all_titles = get_all_job_titles(API_KEY_OPPSLINK)

def clean_description(description):
    """Clean and format the description text."""
    if not description:
        return "Not Disclosed"
    cleaned = html.unescape(description)
    cleaned = cleaned.replace('\r\n', ' ').replace('\n', ' ')
    return ' '.join(cleaned.split()).strip()

def fetch_data(payload, page):
    for attempt in range(5):
        response = httpx.post(EU_API_URL, headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {EU_API_KEY}",
        }, json=payload)

        if response.status_code == 200:
            return response.json().get("notices", [])
        elif response.status_code == 429:
            time.sleep(2 ** attempt)
        else:
            break
    return []

# Step 2: Scrape and filter EU contracts
all_eu_contracts = []
for page in range(1, 11):
    payload = {
        "query": "(classification-cpv IN (66171000 73100000 73200000 73300000 75211200 79411000 90713000)) AND (notice-type IN (qu-sy pin-cfc-standard subco cn-desg cn-social cn-standard pin-cfc-social))  SORT BY publication-number DESC",
        "page": page,
        "limit": 50,
        "fields": [
            "classification-cpv", "BT-24-Procedure", "BT-27-Procedure", "BT-27-Part", "BT-27-Lot",
            "publication-number", "BT-5141-Procedure", "BT-5141-Part", "BT-5141-Lot",
            "BT-5071-Procedure", "BT-5071-Part", "BT-5071-Lot", "BT-727-Procedure",
            "BT-727-Part", "BT-727-Lot", "place-of-performance", "procedure-type",
            "contract-nature", "buyer-name", "buyer-country", "publication-date",
            "deadline-receipt-request", "notice-title", "official-language", "notice-type",
            "links"
        ],
        "validation": False,
        "scope": "ACTIVE",
        "language": "EN"
    }

    new_contracts = fetch_data(payload, page)

    for contract in new_contracts:
        title = contract.get("notice-title", {}).get("eng", "")
        if not title or title in all_titles:
            continue

        official_languages = ', '.join([lang['label'] for lang in contract.get("official-language", [])])
        if 'English' not in official_languages:
            continue

        deadline = contract.get("deadline-receipt-request", [])
        deadline_date = deadline[0] if deadline else "2050-01-01T00:00:00Z"

        country = ', '.join({place['label'] for place in contract.get("place-of-performance", [])})
        client_name = contract.get("buyer-name", {}).get("eng", [""])[0] or "Not Disclosed"
        description = clean_description(contract.get("BT-24-Procedure", {}).get("eng", ""))
        cpv_codes = ', '.join([c["value"] for c in contract.get("classification-cpv", [])])
        value = contract.get("BT-27-Procedure", "") or "Unavailable"
        link = contract.get("links", {}).get("html", {}).get("ENG", "")

        all_eu_contracts.append({
            "title": title,
            "deadline": deadline_date,
            "country": country,
            "client": client_name,
            "description": description[:2000],
            "cpv_codes": cpv_codes,
            "value": value,
            "link": link
        })

    time.sleep(5)

print(f"✅ {len(all_eu_contracts)} contracts ready for processing")

# Step 3: Upload to OppsLink
for contract in all_eu_contracts:
    title = contract['title']
    if title in all_titles:
        continue

    client_name = contract['client']
    client_key = client_name.lower()

    employer_id = employers_dict.get(client_key)
    if employer_id is None:
        print(f"🔍 Employer '{client_name}' not found. Attempting to create...")
        employer_id = create_employer(client_name)
        if employer_id:
            employers_dict[client_key] = employer_id
        else:
            print(f"🚫 Skipping contract '{title}' — failed to create employer.")
            continue


    # Format deadline
    try:
        formatted_deadline = datetime.fromisoformat(contract["deadline"]).strftime("%Y-%m-%d")
    except:
        formatted_deadline = "2025-12-31"

    job_data = {
        "api_key": API_KEY_OPPSLINK,
        "active": 1,
        "featured": 0,
        "title": contract['title'],
        "employer_id": employer_id,
        "description": contract['description'],
        "how_to_apply": "contact@example.com",
        "activation_date": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
        "expiration_date": formatted_deadline,
        "location": contract['country'] or "EU",
        "custom_fields": [
            {"name": "Contract Value", "value": contract['value']},
            {"name": "Contract Link", "value": contract['link']},
            {"name": "CPV Codes", "value": contract['cpv_codes']},
            {"name": "Application deadline", "value": formatted_deadline, "type": "date"}
        ]
    }

    success = create_job(job_data)
    if success:
        all_titles.add(contract['title'])

print("✅ EU contracts uploaded to OppsLink")


Starting to collect all employers...


Processed page 1 - found 100 employers (total: 100)


Processed page 2 - found 100 employers (total: 200)


Processed page 3 - found 100 employers (total: 300)


Processed page 4 - found 100 employers (total: 400)


...

Processed page 3 - found 87 jobs (total: 287)

Finished! Collected 287 unique titles from 287 jobs across 3 pages


✅ 3 contracts ready for processing


❌ Failed to post job Ireland – IT services: consulting, software development, Internet and support – Provision of an ICT Data Protection Compliance Software Solution for the administration of the Data Protection, Freedom of Information and Protected Disclosures Functions (HSE ref. 25027): 400 - {"errors":{"title":["The title may not be greater than 255 characters."]}}


❌ Failed to post job Italy – Research and experimental development services – Pre-commercial procurement of a research and development project of digital solutions and services for the socio-economic development of peripheral territorial communities through the creation and enhancement of cultural tourism destinations (CUP: C53D24000190007- CIG: B6937EED1F): 400 - {"errors":{"title":["The title may not be greater than 255 characters."]}}


❌ Failed to post job Ireland – Services furnished by business, professional and specialist organisations – ComReg T20327. Request for Tenders(RFT) to establish a Multi Supplier Framework Agreement for the provision of Services to support a review of competition in the market to the ComReg: 400 - {"errors":{"title":["The title may not be greater than 255 characters."]}}
✅ EU contracts uploaded to OppsLink


In [8]:
import httpx
import json
import time
from datetime import datetime, timezone
import html
import re
import requests

# Constants
API_KEY_OPPSLINK = "37c597e7bb52d26099ede8b8aa43b270"
HEADERS = {"Content-Type": "application/json"}
EU_API_KEY = "844692394d814535a9ed3a37f0ab01e1"
EU_API_URL = "https://tedweb.api.ted.europa.eu/private-search/api/v1/notices/search"

# Step 1: Get all employers and job titles from OppsLink
employers_dict, employers_list = get_all_employers(API_KEY_OPPSLINK)
all_titles = get_all_job_titles(API_KEY_OPPSLINK)

def clean_description(description):
    """Clean and format the description text."""
    if not description:
        return "Not Disclosed"
    cleaned = html.unescape(description)
    cleaned = cleaned.replace('\r\n', ' ').replace('\n', ' ')
    return ' '.join(cleaned.split()).strip()

def fetch_data(payload, page):
    for attempt in range(5):
        response = httpx.post(EU_API_URL, headers={
            "Content-Type": "application/json",
            "Authorization": f"Bearer {EU_API_KEY}",
        }, json=payload)

        if response.status_code == 200:
            return response.json().get("notices", [])
        elif response.status_code == 429:
            time.sleep(2 ** attempt)
        else:
            break
    return []

# Step 2: Scrape and filter EU contracts
all_eu_contracts = []
for page in range(1, 11):
    payload = {
        "query": "(classification-cpv IN (66171000 73100000 73200000 73300000 75211200 79411000 90713000)) AND (notice-type IN (qu-sy pin-cfc-standard subco cn-desg cn-social cn-standard pin-cfc-social))  SORT BY publication-number DESC",
        "page": page,
        "limit": 50,
        "fields": [
            "classification-cpv", "BT-24-Procedure", "BT-27-Procedure", "BT-27-Part", "BT-27-Lot",
            "publication-number", "BT-5141-Procedure", "BT-5141-Part", "BT-5141-Lot",
            "BT-5071-Procedure", "BT-5071-Part", "BT-5071-Lot", "BT-727-Procedure",
            "BT-727-Part", "BT-727-Lot", "place-of-performance", "procedure-type",
            "contract-nature", "buyer-name", "buyer-country", "publication-date",
            "deadline-receipt-request", "notice-title", "official-language", "notice-type",
            "links"
        ],
        "validation": False,
        "scope": "ACTIVE",
        "language": "EN"
    }

    new_contracts = fetch_data(payload, page)

    for contract in new_contracts:
        full_title = contract.get("notice-title", {}).get("eng", "")
        if not full_title:
            continue

        # Split title to retain first and last parts (e.g., "Country – Services – Description" → "Country – Description")
        if " – " in full_title:
            parts = full_title.split(" – ")
            if len(parts) >= 2:
                title = f"{parts[0]} – {parts[-1]}"
            else:
                title = full_title
        else:
            title = full_title

        if title in all_titles:
            print(f"⏩ Skipping duplicate title: {title}")
            continue

        official_languages = ', '.join([lang['label'] for lang in contract.get("official-language", [])])
        if 'English' not in official_languages:
            continue

        deadline = contract.get("deadline-receipt-request", [])
        deadline_date = deadline[0] if deadline else "2050-01-01T00:00:00Z"

        country = ', '.join({place['label'] for place in contract.get("place-of-performance", [])})
        client_name = contract.get("buyer-name", {}).get("eng", [""])[0] or "Not Disclosed"
        description = clean_description(contract.get("BT-24-Procedure", {}).get("eng", ""))
        cpv_codes = ', '.join([c["value"] for c in contract.get("classification-cpv", [])])
        value = contract.get("BT-27-Procedure", "") or "Unavailable"
        link = contract.get("links", {}).get("html", {}).get("ENG", "")

        all_eu_contracts.append({
            "title": title,
            "deadline": deadline_date,
            "country": country,
            "client": client_name,
            "description": description[:2000],
            "cpv_codes": cpv_codes,
            "value": value,
            "link": link
        })

    time.sleep(5)

print(f"✅ {len(all_eu_contracts)} contracts ready for processing")

# Step 3: Upload to OppsLink
for contract in all_eu_contracts:
    title = contract['title']
    if title in all_titles:
        print(f"⏩ Skipping duplicate title: {title}")
        continue

    client_name = contract['client']
    client_key = client_name.lower()

    employer_id = employers_dict.get(client_key)
    if employer_id is None:
        print(f"🔍 Employer '{client_name}' not found. Attempting to create...")
        employer_id = create_employer(client_name)
        if employer_id:
            employers_dict[client_key] = employer_id
        else:
            print(f"🚫 Skipping contract '{title}' — failed to create employer.")
            continue

    try:
        formatted_deadline = datetime.fromisoformat(contract["deadline"]).strftime("%Y-%m-%d")
    except:
        formatted_deadline = "2025-12-31"

    job_data = {
        "api_key": API_KEY_OPPSLINK,
        "active": 1,
        "featured": 0,
        "title": title,
        "employer_id": employer_id,
        "description": contract['description'],
        "how_to_apply": "contact@example.com",
        "activation_date": datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S"),
        "expiration_date": formatted_deadline,
        "location": contract['country'] or "EU",
        "custom_fields": [
            {"name": "Contract Value", "value": contract['value']},
            {"name": "Contract Link", "value": contract['link']},
            {"name": "CPV Codes", "value": contract['cpv_codes']},
            {"name": "Application deadline", "value": formatted_deadline, "type": "date"}
        ]
    }

    success = create_job(job_data)
    if success:
        all_titles.add(title)

print("✅ EU contracts uploaded to OppsLink")


Starting to collect all employers...


Processed page 1 - found 100 employers (total: 100)


Processed page 2 - found 100 employers (total: 200)


Processed page 3 - found 100 employers (total: 300)


Processed page 4 - found 100 employers (total: 400)


...

⏩ Skipping duplicate title: Belgium – Framework contract for the provision of services to support the implementation of the Single European Sky policy
⏩ Skipping duplicate title: Norway – Dynamic procurement scheme for biology and popular science assistance.
⏩ Skipping duplicate title: Ireland – Consultancy Services Ballycasey Campus
⏩ Skipping duplicate title: Netherlands – CKIC-FIN-001
⏩ Skipping duplicate title: Ireland – CIE Board Advisory Services Qualification System
⏩ Skipping duplicate title: Norway – Attitude survey on ethnic and religious minorities 2025-2027
⏩ Skipping duplicate title: Ireland – Interreg NWE Financial Management Consultancy Tender
⏩ Skipping duplicate title: Ireland, Greece – Interreg NWE Financial Management Consultancy
⏩ Skipping duplicate title: Norway – Dynamic purchasing system CLIMATE, ENVIRONMENT AND NATURE


⏩ Skipping duplicate title: Norway – Framework agreement for "Art in numbers"
⏩ Skipping duplicate title: Germany – 81314138-Comprehensive Gender Responsive Business Approaches Services for SMEs in Jordan
⏩ Skipping duplicate title: Sweden – Central Evaluation Ukraine
⏩ Skipping duplicate title: Denmark – Evaluation of the Danish Energy Partnership Programmes
⏩ Skipping duplicate title: Ireland – HRB 2025 Open Research Publishing
⏩ Skipping duplicate title: Ireland – ComReg T20327. Request for Tenders(RFT) to establish a Multi Supplier Framework Agreement for the provision of Services to support a review of competition in the market to the ComReg
⏩ Skipping duplicate title: Malta – CT7001/2024 - Dynamic Purchasing System for the Provision of Specialised Resources for The Malta Digital Innovation Authority (MDIA) (Eu Funded)
⏩ Skipping duplicate title: Norway – Framework agreement for consultancy services for Education and Research in development contexts
⏩ Skipping duplicate title: Ire

⏩ Skipping duplicate title: Norway – Dynamic purchasing system - Consultancy consultancy services in the technical sector for 6 municipalities in Telemark.
⏩ Skipping duplicate title: Ireland – Professional Services for an independent interim and final external evaluation of Developing Irish Sea Cooperation (DISC)
⏩ Skipping duplicate title: Norway – Disposal of pipelines, cables and other infrastructure on the Norwegian continental shelf.
⏩ Skipping duplicate title: Norway – Communication services, framework agreement
⏩ Skipping duplicate title: Sweden – National Team Leader, EU Accession Support and Administrative support for project in Ukraine
⏩ Skipping duplicate title: Norway – 25/00180 Procurement of a follow-up and result evaluation of Bo safe home reform - The Norwegian Directorate of Health
⏩ Skipping duplicate title: Switzerland – Project UR_01132-01 for the implementation of the ‘Circular Electronics initiative (CEI)’ Programme (mid-2025 to mid-2029)
⏩ Skipping duplicate tit

✅ 1 contracts ready for processing


❌ Failed to post job Italy – Pre-commercial procurement of a research and development project of digital solutions and services for the socio-economic development of peripheral territorial communities through the creation and enhancement of cultural tourism destinations (CUP: C53D24000190007- CIG: B6937EED1F): 400 - {"errors":{"title":["The title may not be greater than 255 characters."]}}
✅ EU contracts uploaded to OppsLink
